# Video enrichment system for pets

### John Burt
### June 2020

### Notebook purpose:

I'm building a PC based system to provide video enrichment to bored pets. The pet could be any animal that can trigger a proximity sensor and which might be entertained by video/audio playback of interesting media content.

The enrichment model presents video as an intrinsic reward. No operant training is required but the pet must be interested enough in the video stimuli to approach the system and trigger playback. 

Video content is hand chosen Youtube video URLs, stored in a csv file. The video content is tagged based on the search term that found the video. This can be used as a category label.


### Model operant behavior:

- Playback triggered by proximity detection by IR sensor.
- Video selected for playback is randomly chosen from a playlist.
- Playback is stopped if pet doesn't trigger IR sensor before stop_time secs.
- If playback stops, another video is randomly selected for next playback.


- while loop:

    - poll proximity switch state
      
    - if proximity switch state == ON (pet in proximity to switch):
        - if not playing:
            - randomly select a new video
            - start playing video
                        
    - else if proximity switch == OFF (pet leaves):
        - if off longer than stop_time:
            - stop playing 
            - log playback info


In [1]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')
# %matplotlib inline
# from matplotlib import pyplot as plt
# import matplotlib
# matplotlib.style.use('ggplot')

from os import path
import pandas as pd
import numpy as np
import time 

import vlc
import pafy

import serial
import serial.tools.list_ports


## IR proximity sensor setup & communication

This code interfaces with an IR rangefinder circuit I designed for testing bird preferences. The device connects to a PC via USB as a 115k Baud serial port. It reads analog range data from two IR sensors and outputs the range values when up or down thresholds are exceeded (that is, it acts as a switch sensor that also outputs range info). Up/down threshold levels, which determine the trigger distance, are set by sending serial text commands to the device. The device also has four LED outputs, two for each IR sensor, which can be turned on or off via serial commands.

#### IR sensor output looks like this:
- "aaaa,bbbb\r\n"
    - where a and b are integer values indicating IR level for sensors 1 & 2. Higher values = closer object

#### This is the help message printed if you type 'H' in a terminal window connected to the device:

<code>Help for operant device #6:
  H = get this help message
  N = get this device serial number
  I = get current IR sensor levels
  L,n,n,n,n = set LEDs 0=off,1=on, order: P1-L1, P1-L2, P2-L1, P2-L2
  D,n = set down (high to low) IR trigger threshold
  U,n = set up (low to high) IR trigger threshold
  T = enter test mode
  E = enter experiment mode (default)
</code>

Note: when sending a command, you must end the string w/ "\r\n"

In [2]:
def get_IR_comport_id():
    """Find the com port id for the IR sensor"""
    ports = list(serial.tools.list_ports.comports())
    for p in ports:
        if 'Silicon Labs CP210x USB to UART Bridge' in p[1]:
            return p[0]
    return None

def send_IR_command(ser, cmd):
    """Send a command to the IR proximity switch"""
    ser.write(('%s\r\n'%(cmd)).encode('utf-8'))
    
def switch_init(port, IR_threshold):
    baudrate = 115200
    ser = serial.Serial(port, baudrate, timeout=0)
    send_IR_command(ser, 'u,%d'%(IR_threshold))
    send_IR_command(ser, 'd,%d'%(IR_threshold))
    return ser

def set_LED_state(ser, l1a, l1b, l2a, l2b):
    """Set IR proximity switch LED states"""
    # Note: l1a & l2a aren't working on the device I'm using
    send_IR_command(ser, 'L,%d,%d,%d,%d'%(l1a,l1b,l2a,l2b))

def parse_proximity_levels(instr):
    """Convert 'nn,nn' string to int state levels """
    try:
        s1 = int(instr.split(',')[0])
        s2 = int(instr.split(',')[1])
        return s1,s2
    except:
        return 0,0
    
def get_switch_state(oldstate, on_thresh, off_thresh, instr):
    """determine current switch state,
      return whether state has changed, and current state"""
    # convert "num,num" str into int values
    s1,s2 = parse_proximity_levels(instr)
    # bypass if error parsing
    if (s1!=0) & (s2!=0):
        newstate = oldstate.copy()
        if s1 > on_thresh: newstate[0] = 1
        elif s1 < off_thresh: newstate[0] = 0
        if s2 > on_thresh: newstate[1] = 1
        elif s2 < off_thresh: newstate[1] = 0
        return newstate!=oldstate, newstate
    return False, oldstate

def check_IR_switch(ser, sw_state, on_thresh, off_thresh):
    """Poll port for device output, process it & return switch state"""
    error = False
    changed = False
    # if incoming bytes are waiting to be read from the 
    #  serial input buffer
    if ser.in_waiting > 0: 
        #read the bytes and convert from binary array to ASCII
        while ser.in_waiting > 0:
            data_str = ser.readline().decode('ascii') 
        # update switch on/off state, check for change
        changed, newstate = get_switch_state(sw_state, 
                                         on_thresh, 
                                         off_thresh,
                                         data_str)
        # update switch state
        if changed == True:
            # sync LED state w/ switch state
            # set_LED_state(ser, 0, newstate[0], 0, newstate[1])
            sw_state = newstate
                
    return error, changed, sw_state 

## Video playback control

For video playback, I'm using a VLC player window, streaming a selected Youtube video. I'm also using a module called pafy to generate the ideal video URL for playback.

In [15]:
def video_init():
    return vlc.Instance()

def video_play(instance, url, starttime=0, volume=0):
    """Play Youtube video at URL"""
    video = pafy.new(url)
    best = video.getbest()
    mediaplayer = instance.media_player_new(uri=best.url)
    mediaplayer.audio_set_volume(volume)
    mediaplayer.set_time(starttime*1000)
    mediaplayer.set_fullscreen(1)
    mediaplayer.play()
    return mediaplayer    

def video_stop(media):
    """Stop playing the video, return stop position in video"""
    if media is not None:
        stoptime = int(media.get_time()/1000)
        media.stop()
        return stoptime
    return 0

def next_video_index(sampindex, current=None):
    """Select increment to the next value in the shuffled sample index list.
        If the index get to list end, then reshuffle the sample index list & 
          return the first entry. """
    if (current == None) | (current >= len(sampindex)-1):
        np.random.shuffle(sampindex)
        return sampindex, 0
    else:
        return sampindex, current+1


## Playback event logging

In [9]:
def log_playback(logpath, pl_info, logtime, play_dur, trigger_count, model_ver):
    try:
        if not path.exists(logpath):
            with open(logpath, 'w+') as file:
                file.write('time\tplay_dur\ttrigger_count\tmodel_ver\tsearch_term\ttitle\turl\trating\tvideo_dur\tvideo_start_pos\n')

        with open(logpath, 'a+') as file:
            file.write('%s\t%1.2f\t%d\t%s\t%s\t%s\t%s\t%1.2f\t%d\t%d\n'%(
                time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(logtime)),
                play_dur,
                trigger_count,
                model_ver,
                pl_info.search_term,
                pl_info.title.replace('\t',''),
                pl_info.url,
                pl_info.rating,
                pl_info.duration,
                pl_info.position,
            ))
    except:
        print('Error: logfile write access denied')


## The sensor / playback loop

In [14]:
playlistdir = './playlists/'
playlistfile = 'youtube_videos_pl.csv'
playlistpath = playlistdir + playlistfile

logdir = './logs/'
logfile = 'playback_log_%s.csv'%(time.strftime('%y%m%d_%H%M%S'))
logpath = logdir + logfile

model_ver = '1.0'

port = get_IR_comport_id()
if port is None:
    print('Error: IR proximity sensor not connected, exiting')
    raise KeyboardInterrupt

df = pd.read_csv(playlistpath)
    
# IR switch configs
test_switch = False
IR_threshold = 100
on_thresh = IR_threshold + 25
off_thresh =  IR_threshold - 25
check_time = 0.5
trigger_switch = 0
# state variables
lastcheck = 0
sw_state = [0,0]
trigger_count = 0

# video configs
stop_time = 20 # time video plays after last switch on before stopping 
# state variables
playing = False
cur_pos = 0
stop_pos = 0
sampidx = None
last_on_time = 0
media = None
volume = 50

# shuffle the playlist
sampindex = np.arange(df.shape[0])
np.random.shuffle(sampindex)

# set up IR proximity switch
ser = switch_init(port, IR_threshold)

# set up video playback system
vlc_instance = video_init()

quit = False
while not quit:
    try:
        # check switch device, handle message, return state
        quit, changed, sw_state = check_IR_switch(
            ser, sw_state, on_thresh, off_thresh)
        
        # set LED
        if changed == True:
            changed = False
            set_LED_state(ser,0,sw_state[trigger_switch],0,0)
            if sw_state[trigger_switch] == 1:
                trigger_count += 1

        # switch is on/triggered
        if sw_state[trigger_switch] == 1:
            # if not playing, start playing a new video
            if playing != True:
                # select next video
                sampindex, cur_pos = next_video_index(sampindex, cur_pos)
                sampidx = sampindex[cur_pos]
                if test_switch != True: 
                    media = video_play(vlc_instance, df.url.iloc[sampidx], 
                                       starttime=df.position.iloc[sampidx], 
                                       volume=volume)
                playing = True
                play_start = time.time()
                # TODO: deal w/ video ending
            last_on_time = time.time()

        # else if switch == off
        else:
            # if playing and past stopping time w/ no trigger, then stop playing
            if ((playing == True) & 
                (time.time() - last_on_time > stop_time)):
                if test_switch != True: 
                    stop_pos = video_stop(media)
                # be sure index is valid
                if sampidx is not None:
                    # set next play time for this video to be current position
                    next_start_pos = stop_pos
                    # reset startpos to 0
                    if next_start_pos + stop_time > df.duration.iloc[sampidx]:
                        next_start_pos = 0
                    df.position.iloc[sampidx] = next_start_pos
                # store last off time this video
                playing = False
                
                # total_duration = now - play_start
                # save playback info: time, category, title, total duration, sw_on_dur
                # sw_on_dur = []
                log_playback(logpath, df.iloc[sampidx], time.time(), time.time()-play_start, 
                             trigger_count, model_ver)
                trigger_count = 0

        # trigger an IR level report every check_time seconds.
        #  This catches occasional missed switch threshold changes.
        if time.time() > lastcheck + check_time:
            send_IR_command(ser, 'i')
            lastcheck = time.time()

        time.sleep(0.01) 
        
    # keep track of video here: check if done playing
    except:
        quit = True
        
print('\nMonitoring and playback stopped')
ser.close()
if test_switch != True: 
    video_stop(media)

KeyboardInterrupt: 

In [13]:
# this is in case you need to manually close the port and stop the video
ser.close()
video_stop(media)

0